In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_api")

#wandb.login(key=wandb_api)
! wandb login $wandb_api

In [ ]:
# Установка необходимых библиотек (актуальные версии на декабрь 2024)
!pip install -q transformers>=4.47.0
!pip install -q peft>=0.13.0
!pip install -q trl>=0.12.0
!pip install -q bitsandbytes>=0.44.0
!pip install -q accelerate>=1.1.0
!pip install -q datasets>=3.1.0
!pip install -q liger-kernel>=0.4.0  # Для оптимизации обучения


In [ ]:
# Импорты
from accelerate import notebook_launcher
import json
import torch
import pandas as pd
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
import shutil
import os
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

print(f" CUDA доступен: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Загрузка конфигурации
# Замените '/kaggle/input/your-dataset' на путь к вашему dataset в Kaggle
DATASET_PATH = "/kaggle/input/dataset-for-llm"  # Измените на ваш dataset

config_file = Path(DATASET_PATH) / "config.json"
with open(config_file, "r", encoding="utf-8") as f:
    config = json.load(f)

print(" Конфигурация загружена:")
print(f"   Модель: {config['model_config']['base_model']}")
print(f"   Обучающих примеров: {config['dataset_info']['train_samples']}")
print(f"   Тестовых примеров: {config['dataset_info']['test_samples']}")
print(f"   LoRA rank: {config['training_config']['lora_r']}")

In [ ]:
# Загрузка данных
def load_jsonl(file_path):
    """Загружает данные из JSONL файла"""
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))
    return data


# Загружаем обучающие и тестовые данные
train_file = Path(DATASET_PATH) / "train_data_clear.jsonl"
test_file = Path(DATASET_PATH) / "test_data_clear.jsonl"

train_data = load_jsonl(train_file)
test_data = load_jsonl(test_file)

print(f"Загружено:")
print(f"   Обучающих: {len(train_data)}")
print(f"   Тестовых: {len(test_data)}")

# Проверка формата данных
print(f"\n Проверка формата данных:")
sample = train_data[0]
print(f"   Ключи в данных: {list(sample.keys())}")
print(f"   Пример input: {sample['input'][:100]}...")
print(f"   Пример output: {sample['output']}")
print(f"   Длина поля 'text': {len(sample['text'])} символов")

# Проверяем, что поле 'text' содержит правильный формат для обучения
if (
    "<s>system" in sample["text"]
    and "<s>user" in sample["text"]
    and "<s>assistant" in sample["text"]
):
    print("Данные в правильном формате ChatML для обучения")
else:
    print("Данные НЕ в правильном формате ChatML!")

In [ ]:
# Создание папок для результатов
output_dir = "./hashtag_lora_model"
merged_dir = "./merged_hashtag_model"
final_dir = "./final_model_for_ollama"

os.makedirs(output_dir, exist_ok=True)
os.makedirs(merged_dir, exist_ok=True)
os.makedirs(final_dir, exist_ok=True)

print(f"📁 Папки созданы в Kaggle:")
print(f"   LoRA модель: {output_dir}")
print(f"   Объединенная модель: {merged_dir}")
print(f"   Финальная модель: {final_dir}")

In [ ]:
#  КРИТИЧЕСКАЯ ПРОВЕРКА: Правильность данных для обучения генерации хештегов

print("Детальная проверка данных для обучения генерации хештегов:")
print("=" * 70)

# Проверяем первые несколько примеров
for i, sample in enumerate(train_data[:3]):
    print(f"\n📋 Пример {i+1}:")
    print(f"   Input (новость): {sample['input'][:80]}...")
    print(f"   Output (хештеги): {sample['output']}")

    # Проверяем поле 'text' - это то, на чем будет обучаться модель
    text_parts = sample["text"].split("<s>")
    for part in text_parts:
        if part.startswith("system"):
            system_msg = part.replace("system\n", "").replace("</s>", "")
            print(f"   System: {system_msg[:50]}...")
        elif part.startswith("user"):
            user_msg = part.replace("user\n", "").replace("</s>", "")
            print(f"   User: {user_msg[:50]}...")
        elif part.startswith("assistant"):
            assistant_msg = part.replace("assistant\n", "").replace("</s>", "")
            print(f"   Assistant: {assistant_msg}")

            # КРИТИЧЕСКАЯ ПРОВЕРКА: Совпадают ли хештеги?
            if assistant_msg.strip() == sample["output"].strip():
                print(f"   Хештеги совпадают - обучение будет правильным!")
            else:
                print(f"   ОШИБКА: Хештеги НЕ совпадают!")
                print(f"      В 'text': '{assistant_msg.strip()}'")
                print(f"      В 'output': '{sample['output'].strip()}'")

print("\n" + "=" * 70)
print("ВЫВОД: Модель будет обучаться генерировать хештеги из поля 'output'")
print("Формат обучения: ChatML с системным промптом + новость + хештеги")
print("Это правильная настройка для генерации хештегов!")

In [ ]:
from transformers import logging


logging.enable_progress_bar()
logging.set_verbosity_info()

In [ ]:
import wandb


def traning_this():

    wandb.init(
        project="llm-hashtag-generation",
        name=f"lora-r{config['training_config']['lora_r']}-lr{config['training_config']['learning_rate']}",
        config=config["training_config"],
    )

    # Настройка квантизации для экономии памяти
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    # Загрузка модели и токенизатора
    model_name = config["model_config"]["base_model"]
    print(f"Загрузка модели: {model_name}")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
    )

    model = prepare_model_for_kbit_training(model)

    # Настройка LoRA
    lora_config = LoraConfig(
        r=config["training_config"]["lora_r"],
        lora_alpha=config["training_config"]["lora_alpha"],
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    print("Модель и LoRA настроены")

    train_texts = [item["text"] for item in train_data]
    test_texts = [item["text"] for item in test_data]

    train_dataset = Dataset.from_dict({"text": train_texts})
    test_dataset = Dataset.from_dict({"text": test_texts})

    training_args = SFTConfig(
        output_dir=output_dir,
        num_train_epochs=config["training_config"]["epochs"],
        per_device_train_batch_size=config["training_config"]["batch_size"],
        per_device_eval_batch_size=config["training_config"]["batch_size"],
        gradient_accumulation_steps=4,  # Для стабильности
        optim="paged_adamw_32bit",
        save_steps=100,
        logging_steps=10,
        learning_rate=config["training_config"]["learning_rate"],
        weight_decay=0.001,
        fp16=True,
        bf16=False,
        max_grad_norm=0.3,
        max_steps=-1,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="cosine",
        report_to="wandb",
        eval_strategy="steps",
        eval_steps=50,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        run_name=wandb.run.name,
        # SFT-специфичные параметры
        max_seq_length=512,
        dataset_text_field="text",
        packing=False,
        use_liger_kernel=False,
        remove_unused_columns=False,
        # dataloader_num_workers = 0
    )

    # Создание тренера с правильными параметрами для TRL 0.12.0
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        peft_config=lora_config,
        args=training_args,
        processing_class=tokenizer,
    )

    print("✅ Тренер настроен с актуальными параметрами")

    # Обучение модели
    trainer.train()

    # Сохранение модели и токенизатора
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

    # Создание объединенной модели (LoRA + базовая)
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map="cpu", trust_remote_code=True
    )
    merged_model = PeftModel.from_pretrained(base_model, output_dir)
    merged_model = merged_model.merge_and_unload()

    # Сохранение объединенной модели
    merged_model.save_pretrained(os.path.join(output_dir, "merged_model"))
    tokenizer.save_pretrained(os.path.join(output_dir, "merged_model"))

    return {
        "trainer": trainer,
        "model": merged_model,
        "tokenizer": tokenizer,
        "output_dir": output_dir,
        "merged_model_path": os.path.join(output_dir, "merged_model"),
    }


# Запуск обучения
results = notebook_launcher(traning_this, num_processes=1)

In [ ]:
trainer = results["trainer"]
model = results["model"]
output_dir = results["output_dir"]
merged_model_path = results["merged_model_path"]

In [ ]:
# Создание финальной модели для Ollama
print("Подготовка модели для Ollama...")

# Копируем объединенную модель в финальную папку
shutil.copytree(merged_dir, final_dir, dirs_exist_ok=True)

# Создаем Modelfile для Ollama
modelfile_content = '''FROM ./

SYSTEM """Ты эксперт по генерации хештегов для новостей, специально обученный на российских новостных данных.

Твоя задача - анализировать новостные тексты и генерировать релевантные хештеги на русском языке.

Правила генерации:
1. Создавай 3-5 хештегов для каждой новости
2. Хештеги должны быть конкретными и описательными
3. Используй только русский язык
4. Не добавляй символ # в ответе
5. Разделяй хештеги запятыми
6. Фокусируйся на ключевых темах и событиях

Отвечай только хештегами, без дополнительных комментариев."""

PARAMETER temperature 0.1
PARAMETER top_p 0.9
PARAMETER repeat_penalty 1.1
PARAMETER num_ctx 2048
PARAMETER stop "<s>"
PARAMETER stop "</s>"
'''

with open(f"{final_dir}/Modelfile", "w", encoding="utf-8") as f:
    f.write(modelfile_content)

print(f"✅ Modelfile создан в {final_dir}")

In [ ]:
# Создание README с инструкциями
instructions = f"""# 🚀 Дообученная модель Saiga LLaMA3 для генерации хештегов

## 📊 Информация об обучении
- Базовая модель: {model_name}
- Обучающих примеров: {config['dataset_info']['train_samples']}
- Эпох обучения: {config['training_config']['epochs']}
- LoRA rank: {config['training_config']['lora_r']}
- Версия TRL: 0.12.0+ (декабрь 2024)

## 🔧 Установка в Ollama

1. Скачайте и распакуйте эту папку
2. Перейдите в папку с моделью:
   ```bash
   cd path/to/final_model_for_ollama
   ```

3. Создайте модель в Ollama:
   ```bash
   ollama create saiga-hashtag-pro -f Modelfile
   ```

4. Проверьте установку:
   ```bash
   ollama list
   ```

## 🧪 Тестирование

```bash
ollama run saiga-hashtag-pro "Центральный банк повысил ключевую ставку"
```

## Примеры использования

**Вход:** "Новый iPhone 15 поступил в продажу в России"
**Выход:** "технологии, смартфоны, продажи"

**Вход:** "Сборная России выиграла матч по футболу"
**Выход:** "спорт, футбол, сборная_россии"

## Особенности этой версии
- Обучена на актуальных российских новостях
- Использует современные версии библиотек (TRL 0.12.0+)
- Оптимизирована для экономии памяти
- Правильно настроена для генерации хештегов
- Проверена на правильность обучения

##  Формат обучения
- **Данные**: ChatML формат с системным промптом
- **Поле обучения**: `text` (полный диалог)
- **Целевые хештеги**: из поля `output`
- **Проверка**: Хештеги в `text` и `output` совпадают ✅
"""

with open(f"{final_dir}/README.md", "w", encoding="utf-8") as f:
    f.write(instructions)

print(f"✅ README создан в {final_dir}")